In [1]:
# 감정 분석 FastAPI 웹서버
# 003-03.ipynb에서 저장한 모델을 사용하여 긍정/부정 감정을 분석하는 웹서버

from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import pickle
import re
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import uvicorn
from typing import List, Dict
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI(
    title="감정 분석 API",
    description="리뷰 텍스트의 긍정/부정을 분석하는 API",
    version="1.0.0"
)


2025-09-19 20:19:20.882199: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/aaron/Github/Kmong/Aaron/hg-mldl2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# 요청/응답 모델 정의
class TextRequest(BaseModel):
    text: str
    
class TextBatchRequest(BaseModel):
    texts: List[str]

class SentimentResponse(BaseModel):
    text: str
    prediction: str  # "긍정" or "부정"
    confidence: float  # 신뢰도 (0-1)
    probability: float  # 원시 확률 (0-1)

class BatchSentimentResponse(BaseModel):
    results: List[SentimentResponse]


In [3]:
# 감정 분석 예측기 클래스
class SentimentPredictor:
    """감정 분석 예측기 클래스"""
    
    def __init__(self, model_path: str, vectorizer_path: str, scaler_path: str):
        """
        저장된 모델과 전처리기들을 불러와서 초기화
        
        Args:
            model_path: 저장된 Keras 모델 경로
            vectorizer_path: 저장된 TF-IDF 벡터라이저 경로  
            scaler_path: 저장된 StandardScaler 경로
        """
        self.model_path = model_path
        self.vectorizer_path = vectorizer_path
        self.scaler_path = scaler_path
        
        # 모델과 전처리기 불러오기
        self.load_components()
    
    def load_components(self):
        """모델과 전처리기들을 메모리에 로드"""
        try:
            logger.info("모델 및 전처리기 로딩 중...")
            
            # Keras 모델 로드
            self.model = load_model(self.model_path)
            logger.info(f"모델 로드 완료: {self.model_path}")
            
            # TF-IDF 벡터라이저 로드
            with open(self.vectorizer_path, 'rb') as f:
                self.vectorizer = pickle.load(f)
            logger.info(f"TF-IDF 벡터라이저 로드 완료: {self.vectorizer_path}")
            
            # StandardScaler 로드
            with open(self.scaler_path, 'rb') as f:
                self.scaler = pickle.load(f)
            logger.info(f"표준화 스케일러 로드 완료: {self.scaler_path}")
            
            logger.info("모든 컴포넌트 로드 완료!")
            
        except Exception as e:
            logger.error(f"로딩 중 오류 발생: {e}")
            raise
    
    def preprocess_text(self, text: str) -> str:
        """
        텍스트 전처리 (훈련 시와 동일한 방식)
        """
        if pd.isna(text):
            return ""
        
        # 문자열로 변환
        text = str(text)
        
        # HTML 태그 제거
        text = re.sub(r'<[^>]+>', '', text)
        
        # URL 제거
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        
        # 이메일 제거
        text = re.sub(r'\\S+@\\S+', '', text)
        
        # 특수문자는 공백으로 대체 (한글, 영문, 숫자만 유지)
        text = re.sub(r'[^가-힣a-zA-Z0-9\\s]', ' ', text)
        
        # 연속된 공백을 하나로
        text = re.sub(r'\\s+', ' ', text)
        
        # 앞뒤 공백 제거
        text = text.strip()
        
        return text
    
    def predict(self, text: str) -> tuple:
        """
        텍스트의 감정을 예측
        
        Args:
            text: 예측할 텍스트
            
        Returns:
            tuple: (예측결과, 신뢰도, 원시확률)
        """
        try:
            # 텍스트 전처리
            processed_text = self.preprocess_text(text)
            
            if not processed_text.strip():
                return "부정", 0.5, 0.5  # 빈 텍스트의 경우 기본값
            
            # TF-IDF 벡터화
            text_vector = self.vectorizer.transform([processed_text])
            
            # 정규화
            text_scaled = self.scaler.transform(text_vector.toarray())
            
            # 예측
            prob = self.model.predict(text_scaled, verbose=0)[0][0]
            prediction = "긍정" if prob > 0.5 else "부정"
            confidence = prob if prob > 0.5 else (1 - prob)
            
            return prediction, float(confidence), float(prob)
            
        except Exception as e:
            logger.error(f"예측 중 오류 발생: {e}")
            raise HTTPException(status_code=500, detail=f"예측 중 오류 발생: {str(e)}")
    
    def predict_batch(self, texts: List[str]) -> List[tuple]:
        """
        여러 텍스트를 한번에 예측
        
        Args:
            texts: 예측할 텍스트 리스트
            
        Returns:
            list: [(예측결과, 신뢰도, 원시확률), ...] 형태의 리스트
        """
        results = []
        for text in texts:
            result = self.predict(text)
            results.append(result)
        return results


In [4]:
# 모델 초기화 (서버 시작 시 한 번만 실행)
def initialize_model():
    """저장된 모델 파일들을 찾아서 예측기를 초기화"""
    try:
        # 모델 파일 경로 설정
        model_dir = "saved_models"
        model_name = "sentiment_mlp_model_20250916_193146"  # 003-03.ipynb에서 저장된 모델명
        
        model_path = os.path.join(model_dir, f"{model_name}.keras")
        vectorizer_path = os.path.join(model_dir, f"{model_name}_vectorizer.pkl")
        scaler_path = os.path.join(model_dir, f"{model_name}_scaler.pkl")
        
        # 파일 존재 확인
        if not all(os.path.exists(path) for path in [model_path, vectorizer_path, scaler_path]):
            raise FileNotFoundError("모델 파일들을 찾을 수 없습니다. 003-03.ipynb를 먼저 실행하여 모델을 저장해주세요.")
        
        # 예측기 초기화
        predictor = SentimentPredictor(model_path, vectorizer_path, scaler_path)
        logger.info("모델 초기화 완료!")
        return predictor
        
    except Exception as e:
        logger.error(f"모델 초기화 실패: {e}")
        raise

# 전역 예측기 변수
predictor = None

@app.on_event("startup")
async def startup_event():
    """서버 시작 시 모델 로드"""
    global predictor
    try:
        predictor = initialize_model()
        logger.info("서버 시작 완료!")
    except Exception as e:
        logger.error(f"서버 시작 실패: {e}")
        raise


/var/folders/rb/qh6lstqx6_x1zhdc_b0z8h840000gn/T/ipykernel_39752/1933391706.py:29: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [5]:
# API 엔드포인트들

@app.get("/", response_class=HTMLResponse)
async def root():
    """메인 페이지 - 간단한 웹 인터페이스"""
    html_content = """
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>감정 분석 API</title>
        <style>
            body { font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }
            .container { background: #f5f5f5; padding: 20px; border-radius: 10px; margin: 20px 0; }
            textarea { width: 100%; height: 100px; padding: 10px; border: 1px solid #ddd; border-radius: 5px; }
            button { background: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; }
            button:hover { background: #0056b3; }
            .result { margin-top: 20px; padding: 15px; border-radius: 5px; }
            .positive { background: #d4edda; border: 1px solid #c3e6cb; color: #155724; }
            .negative { background: #f8d7da; border: 1px solid #f5c6cb; color: #721c24; }
            .example { background: #e2e3e5; padding: 10px; margin: 5px 0; border-radius: 3px; cursor: pointer; }
            .example:hover { background: #d1d3d4; }
        </style>
    </head>
    <body>
        <h1>🎭 감정 분석 API</h1>
        <p>리뷰나 댓글의 감정을 분석하여 긍정/부정을 판단합니다.</p>
        
        <div class="container">
            <h3>텍스트 입력</h3>
            <textarea id="textInput" placeholder="분석할 텍스트를 입력하세요..."></textarea>
            <br><br>
            <button onclick="analyzeSentiment()">감정 분석하기</button>
            <div id="result"></div>
        </div>
        
        <div class="container">
            <h3>예시 텍스트 (클릭하여 테스트)</h3>
            <div class="example" onclick="setExample('정말 맛있어요! 최고입니다!')">정말 맛있어요! 최고입니다!</div>
            <div class="example" onclick="setExample('서비스가 별로네요... 실망이에요')">서비스가 별로네요... 실망이에요</div>
            <div class="example" onclick="setExample('가격 대비 괜찮은 것 같아요')">가격 대비 괜찮은 것 같아요</div>
            <div class="example" onclick="setExample('완전 최악이에요. 다시는 안 와요')">완전 최악이에요. 다시는 안 와요</div>
            <div class="example" onclick="setExample('직원들이 친절하고 음식도 좋아요')">직원들이 친절하고 음식도 좋아요</div>
        </div>
        
        <div class="container">
            <h3>API 사용법</h3>
            <p><strong>단일 텍스트 분석:</strong> POST /predict</p>
            <pre>{"text": "분석할 텍스트"}</pre>
            
            <p><strong>배치 분석:</strong> POST /predict/batch</p>
            <pre>{"texts": ["텍스트1", "텍스트2", ...]}</pre>
            
            <p><strong>API 문서:</strong> <a href="/docs">/docs</a></p>
        </div>

        <script>
            function setExample(text) {
                document.getElementById('textInput').value = text;
            }
            
            async function analyzeSentiment() {
                const text = document.getElementById('textInput').value.trim();
                if (!text) {
                    alert('텍스트를 입력해주세요.');
                    return;
                }
                
                try {
                    const response = await fetch('/predict', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json',
                        },
                        body: JSON.stringify({text: text})
                    });
                    
                    const data = await response.json();
                    
                    if (response.ok) {
                        const resultClass = data.prediction === '긍정' ? 'positive' : 'negative';
                        document.getElementById('result').innerHTML = `
                            <div class="result ${resultClass}">
                                <h4>분석 결과</h4>
                                <p><strong>예측:</strong> ${data.prediction}</p>
                                <p><strong>신뢰도:</strong> ${(data.confidence * 100).toFixed(1)}%</p>
                                <p><strong>확률:</strong> ${(data.probability * 100).toFixed(1)}%</p>
                            </div>
                        `;
                    } else {
                        document.getElementById('result').innerHTML = `
                            <div class="result negative">
                                <h4>오류 발생</h4>
                                <p>${data.detail || '알 수 없는 오류가 발생했습니다.'}</p>
                            </div>
                        `;
                    }
                } catch (error) {
                    document.getElementById('result').innerHTML = `
                        <div class="result negative">
                            <h4>네트워크 오류</h4>
                            <p>서버에 연결할 수 없습니다: ${error.message}</p>
                        </div>
                    `;
                }
            }
        </script>
    </body>
    </html>
    """
    return html_content

@app.get("/health")
async def health_check():
    """서버 상태 확인"""
    return {
        "status": "healthy",
        "model_loaded": predictor is not None,
        "message": "감정 분석 API가 정상 작동 중입니다."
    }


In [6]:
# 감정 분석 API 엔드포인트들

@app.post("/predict", response_model=SentimentResponse)
async def predict_sentiment(request: TextRequest):
    """
    단일 텍스트의 감정을 분석합니다.
    
    - **text**: 분석할 텍스트
    
    Returns:
    - **prediction**: "긍정" 또는 "부정"
    - **confidence**: 예측 신뢰도 (0-1)
    - **probability**: 긍정일 확률 (0-1)
    """
    if predictor is None:
        raise HTTPException(status_code=503, detail="모델이 로드되지 않았습니다.")
    
    if not request.text.strip():
        raise HTTPException(status_code=400, detail="빈 텍스트는 분석할 수 없습니다.")
    
    try:
        prediction, confidence, probability = predictor.predict(request.text)
        
        return SentimentResponse(
            text=request.text,
            prediction=prediction,
            confidence=confidence,
            probability=probability
        )
    except Exception as e:
        logger.error(f"예측 중 오류: {e}")
        raise HTTPException(status_code=500, detail="예측 중 오류가 발생했습니다.")

@app.post("/predict/batch", response_model=BatchSentimentResponse)
async def predict_sentiment_batch(request: TextBatchRequest):
    """
    여러 텍스트의 감정을 한번에 분석합니다.
    
    - **texts**: 분석할 텍스트 리스트
    
    Returns:
    - **results**: 각 텍스트에 대한 분석 결과 리스트
    """
    if predictor is None:
        raise HTTPException(status_code=503, detail="모델이 로드되지 않았습니다.")
    
    if not request.texts:
        raise HTTPException(status_code=400, detail="분석할 텍스트가 없습니다.")
    
    if len(request.texts) > 100:  # 배치 크기 제한
        raise HTTPException(status_code=400, detail="한 번에 최대 100개의 텍스트만 처리할 수 있습니다.")
    
    try:
        results = []
        batch_predictions = predictor.predict_batch(request.texts)
        
        for text, (prediction, confidence, probability) in zip(request.texts, batch_predictions):
            results.append(SentimentResponse(
                text=text,
                prediction=prediction,
                confidence=confidence,
                probability=probability
            ))
        
        return BatchSentimentResponse(results=results)
        
    except Exception as e:
        logger.error(f"배치 예측 중 오류: {e}")
        raise HTTPException(status_code=500, detail="배치 예측 중 오류가 발생했습니다.")

@app.get("/model/info")
async def get_model_info():
    """모델 정보를 반환합니다."""
    if predictor is None:
        raise HTTPException(status_code=503, detail="모델이 로드되지 않았습니다.")
    
    return {
        "model_name": "감정 분석 MLP 모델",
        "version": "1.0.0",
        "model_file": os.path.basename(predictor.model_path),
        "vectorizer_file": os.path.basename(predictor.vectorizer_path),
        "scaler_file": os.path.basename(predictor.scaler_path),
        "total_parameters": predictor.model.count_params(),
        "input_features": predictor.model.input_shape[1],
        "classes": ["부정", "긍정"],
        "description": "TF-IDF + MLP 기반 한국어 감정 분석 모델"
    }


In [7]:
# Jupyter 노트북용 서버 실행 함수
import asyncio
import threading
import time
from contextlib import asynccontextmanager

def run_server_jupyter(host: str = "127.0.0.1", port: int = 8000):
    """
    Jupyter 노트북에서 FastAPI 서버를 실행합니다.
    
    Args:
        host: 서버 호스트 (기본값: 127.0.0.1)
        port: 서버 포트 (기본값: 8000)
    """
    print(f"🚀 감정 분석 FastAPI 서버 시작! (Jupyter 모드)")
    print(f"📍 주소: http://{host}:{port}")
    print(f"📖 API 문서: http://{host}:{port}/docs")
    print(f"🎭 웹 인터페이스: http://{host}:{port}")
    print(f"❤️ 상태 확인: http://{host}:{port}/health")
    print()
    print("서버를 중지하려면 Kernel -> Interrupt를 선택하세요.")
    
    # Jupyter에서 실행할 수 있도록 별도 스레드에서 서버 실행
    def run_uvicorn():
        import asyncio
        import uvicorn
        
        # 새로운 이벤트 루프 생성
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
        config = uvicorn.Config(
            app=app,
            host=host,
            port=port,
            log_level="info"
        )
        server = uvicorn.Server(config)
        
        try:
            loop.run_until_complete(server.serve())
        except KeyboardInterrupt:
            print("\n🛑 서버가 중지되었습니다.")
        finally:
            loop.close()
    
    # 백그라운드 스레드에서 서버 실행
    server_thread = threading.Thread(target=run_uvicorn, daemon=True)
    server_thread.start()
    
    try:
        # 메인 스레드는 서버 스레드가 살아있는 동안 대기
        while server_thread.is_alive():
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("\n🛑 서버 중지 요청됨")
    
    return server_thread

# 일반 Python 스크립트용 서버 실행 함수
def run_server_script(host: str = "127.0.0.1", port: int = 8000, reload: bool = False):
    """
    일반 Python 스크립트에서 FastAPI 서버를 실행합니다.
    
    Args:
        host: 서버 호스트 (기본값: 127.0.0.1)
        port: 서버 포트 (기본값: 8000)
        reload: 코드 변경 시 자동 재시작 (기본값: False)
    """
    print(f"🚀 감정 분석 FastAPI 서버 시작!")
    print(f"📍 주소: http://{host}:{port}")
    print(f"📖 API 문서: http://{host}:{port}/docs")
    print(f"🎭 웹 인터페이스: http://{host}:{port}")
    print(f"❤️ 상태 확인: http://{host}:{port}/health")
    print()
    print("서버를 중지하려면 Ctrl+C를 누르세요.")
    
    uvicorn.run(
        app,  # app 객체 직접 전달
        host=host,
        port=port,
        reload=reload,
        log_level="info"
    )

# 환경 감지 및 적절한 실행 함수 선택
def run_server(host: str = "127.0.0.1", port: int = 8000, reload: bool = False):
    """
    환경을 자동 감지하여 적절한 방식으로 서버를 실행합니다.
    """
    try:
        # Jupyter 환경인지 확인
        get_ipython()
        print("🔍 Jupyter 환경 감지됨")
        return run_server_jupyter(host, port)
    except NameError:
        # 일반 Python 스크립트 환경
        print("🔍 Python 스크립트 환경 감지됨")
        return run_server_script(host, port, reload)

# 메인 실행부 (Jupyter에서는 사용하지 않음)
if __name__ == "__main__":
    # 서버 실행 (기본 설정)
    run_server()
    
    # 다른 설정으로 실행하려면:
    # run_server(host="0.0.0.0", port=8080, reload=True)  # 외부 접근 허용, 포트 8080, 자동 재시작


🔍 Jupyter 환경 감지됨
🚀 감정 분석 FastAPI 서버 시작! (Jupyter 모드)
📍 주소: http://127.0.0.1:8000
📖 API 문서: http://127.0.0.1:8000/docs
🎭 웹 인터페이스: http://127.0.0.1:8000
❤️ 상태 확인: http://127.0.0.1:8000/health

서버를 중지하려면 Kernel -> Interrupt를 선택하세요.


INFO:     Started server process [39752]
INFO:     Waiting for application startup.
INFO:__main__:모델 및 전처리기 로딩 중...
INFO:__main__:모델 로드 완료: saved_models/sentiment_mlp_model_20250916_193146.keras
INFO:__main__:TF-IDF 벡터라이저 로드 완료: saved_models/sentiment_mlp_model_20250916_193146_vectorizer.pkl
INFO:__main__:표준화 스케일러 로드 완료: saved_models/sentiment_mlp_model_20250916_193146_scaler.pkl
INFO:__main__:모든 컴포넌트 로드 완료!
INFO:__main__:모델 초기화 완료!
INFO:__main__:서버 시작 완료!
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63646 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:63646 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:63646 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:63646 - "POST /predict HTTP/1.1" 200 OK

🛑 서버 중지 요청됨


# 🚀 FastAPI 감정 분석 웹서버 사용 가이드

## 📋 개요
003-03.ipynb에서 훈련하고 저장한 MLP 감정 분석 모델을 사용하는 FastAPI 웹서버입니다.

## 🛠️ 설치 및 실행

### 1. 필요한 패키지 설치
```bash
pip install fastapi uvicorn tensorflow scikit-learn pandas numpy
```

### 2. 서버 실행 방법

#### 방법 1: Jupyter 노트북에서 직접 실행
위의 모든 셀을 순서대로 실행한 후, 마지막 셀을 실행하면 서버가 시작됩니다.

#### 방법 2: Python 스크립트로 저장하여 실행
노트북을 `.py` 파일로 내보낸 후 터미널에서 실행:
```bash
python sentiment_api.py
```

#### 방법 3: uvicorn으로 직접 실행
```bash
uvicorn main:app --host 127.0.0.1 --port 8000 --reload
```

## 🌐 API 엔드포인트

### 메인 페이지
- **URL**: `http://127.0.0.1:8000/`
- **설명**: 웹 브라우저에서 직접 텍스트를 입력하여 감정 분석 가능

### 단일 텍스트 분석
- **URL**: `POST http://127.0.0.1:8000/predict`
- **요청 형식**:
```json
{
    "text": "정말 맛있어요! 최고입니다!"
}
```
- **응답 형식**:
```json
{
    "text": "정말 맛있어요! 최고입니다!",
    "prediction": "긍정",
    "confidence": 0.999,
    "probability": 0.999
}
```

### 배치 텍스트 분석
- **URL**: `POST http://127.0.0.1:8000/predict/batch`
- **요청 형식**:
```json
{
    "texts": [
        "정말 맛있어요!",
        "서비스가 별로네요...",
        "가격 대비 괜찮아요"
    ]
}
```

### 모델 정보
- **URL**: `GET http://127.0.0.1:8000/model/info`
- **설명**: 로드된 모델의 상세 정보 조회

### 상태 확인
- **URL**: `GET http://127.0.0.1:8000/health`
- **설명**: 서버 및 모델 로드 상태 확인

### API 문서
- **URL**: `http://127.0.0.1:8000/docs`
- **설명**: 자동 생성된 Swagger UI API 문서

## 🧪 테스트 예시

### curl을 사용한 테스트
```bash
# 단일 텍스트 분석
curl -X POST "http://127.0.0.1:8000/predict" \
     -H "Content-Type: application/json" \
     -d '{"text": "정말 맛있어요!"}'

# 상태 확인
curl -X GET "http://127.0.0.1:8000/health"
```

### Python requests를 사용한 테스트
```python
import requests

# 단일 예측
response = requests.post(
    "http://127.0.0.1:8000/predict",
    json={"text": "정말 맛있어요!"}
)
print(response.json())

# 배치 예측
response = requests.post(
    "http://127.0.0.1:8000/predict/batch",
    json={"texts": ["좋아요!", "별로네요..."]}
)
print(response.json())
```

## ⚠️ 주의사항
1. **모델 파일 위치**: `saved_models/` 디렉토리에 003-03.ipynb에서 저장한 모델 파일들이 있어야 합니다.
2. **메모리 사용량**: TensorFlow 모델이 로드되므로 충분한 메모리가 필요합니다.
3. **포트 충돌**: 8000번 포트가 사용 중인 경우 다른 포트를 사용하세요.

## 🔧 문제 해결
- **모델 로드 실패**: 003-03.ipynb를 먼저 실행하여 모델을 저장했는지 확인
- **포트 사용 중**: `run_server(port=8080)` 등으로 다른 포트 사용
- **메모리 부족**: 다른 프로그램을 종료하고 재시도


In [ ]:
# 서버 실행 (이 셀을 실행하면 FastAPI 서버가 시작됩니다)
# 주의: 이 셀을 실행하면 서버가 시작되고 셀이 계속 실행 상태를 유지합니다.
# 서버를 중지하려면 Kernel -> Interrupt를 선택하세요.

print("🔧 서버 실행 준비 중...")
print("📁 모델 파일 확인 중...")

# 모델 파일 존재 확인
model_dir = "saved_models"
model_name = "sentiment_mlp_model_20250916_193146"

model_files = [
    f"{model_name}.keras",
    f"{model_name}_vectorizer.pkl", 
    f"{model_name}_scaler.pkl"
]

missing_files = []
for file in model_files:
    file_path = os.path.join(model_dir, file)
    if os.path.exists(file_path):
        print(f"✅ {file}")
    else:
        print(f"❌ {file}")
        missing_files.append(file)

if missing_files:
    print(f"\n⚠️ 누락된 파일들: {missing_files}")
    print("💡 003-03.ipynb를 먼저 실행하여 모델을 저장해주세요.")
else:
    print(f"\n🎉 모든 모델 파일이 준비되었습니다!")
    print("🚀 FastAPI 서버를 시작합니다...")
    print()
    
    # 서버 실행 (Jupyter 환경 자동 감지)
    try:
        server_thread = run_server(host="127.0.0.1", port=8000)
        print("\n✅ 서버가 백그라운드에서 실행 중입니다!")
        print("🌐 브라우저에서 http://127.0.0.1:8000 에 접속하세요!")
        
    except KeyboardInterrupt:
        print("\n🛑 서버가 중지되었습니다.")
    except Exception as e:
        print(f"\n❌ 서버 실행 중 오류 발생: {e}")
        print("💡 문제 해결 방법:")
        print("   1. 포트가 이미 사용 중인 경우: run_server(port=8080)")
        print("   2. 모델 파일이 없는 경우: 003-03.ipynb를 먼저 실행")
        print("   3. 패키지가 없는 경우: pip install -r requirements.txt")


In [ ]:
# 서버 상태 확인 및 테스트 함수들

def check_server_status(host="127.0.0.1", port=8000):
    """서버가 실행 중인지 확인합니다."""
    import requests
    try:
        response = requests.get(f"http://{host}:{port}/health", timeout=5)
        if response.status_code == 200:
            print(f"✅ 서버가 정상 실행 중입니다! (http://{host}:{port})")
            return True
        else:
            print(f"⚠️ 서버 응답 오류: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ 서버에 연결할 수 없습니다: {e}")
        return False

def test_sentiment_api(text="정말 맛있어요!", host="127.0.0.1", port=8000):
    """감정 분석 API를 테스트합니다."""
    import requests
    try:
        response = requests.post(
            f"http://{host}:{port}/predict",
            json={"text": text},
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"📝 입력: \"{text}\"")
            print(f"🎯 예측: {result['prediction']}")
            print(f"📊 신뢰도: {result['confidence']:.3f}")
            print(f"📈 확률: {result['probability']:.3f}")
            return result
        else:
            print(f"❌ API 오류: {response.status_code}")
            print(f"오류 내용: {response.text}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ API 요청 실패: {e}")
        return None

# 사용 예시:
# check_server_status()  # 서버 상태 확인
# test_sentiment_api("정말 맛있어요!")  # 감정 분석 테스트
